In [1]:
#Default
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def reset(df):
    cols = df.columns
    return df.reset_index()[cols]
def print_counts(df):
    cols = df.columns
    for each in cols:
        print(each)
        print(df[each].value_counts())
        print('______________________________________')
# ~
#Default Ending

import os
from tqdm import tqdm

print('OK !')

OK !


In [2]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import DensePassageRetriever, EmbeddingRetriever, FARMReader, EntityExtractor, QuestionGenerator
from haystack.document_stores import ElasticsearchDocumentStore

import torch

print('OK !')

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


OK !


# Embedding Retrieval

In [3]:
# For EmbeddingRetriever
similarity_type = "cosine"

container_name = "localhost"

er_co = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_er_bigram_collocation",
                                           similarity=similarity_type, embedding_dim=768)

er_200 = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_er_200",
                                           similarity=similarity_type, embedding_dim=768)

er_qg = ElasticsearchDocumentStore(host=container_name, port = "9200", index="production_er_question_generation",
                                           similarity=similarity_type, embedding_dim=768)

In [4]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor

converter = PDFToTextConverter(remove_numeric_tables=False, valid_languages = ["en"])

print('OK !')

OK !


pdftotext version 4.03 [www.xpdfreader.com]
Copyright 1996-2021 Glyph & Cog, LLC


# Set document_store 

In [5]:
document_store = er_co
document_store_qg_input = er_200
document_store_qg = er_qg

# Process Raw PDF

In [6]:
def clean_content(content):
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('..','.')
    content = content.replace('\n',' ')
    content = content.replace('\n',' ')
    content = " ".join(content.split())
    return content

print('OK !')

OK !


In [7]:
# import pickle

# with open('data/valid_file_list.pickle', 'rb') as handle:
#     file_list = pickle.load(handle)
    
# len(file_list)

In [8]:
# len(file_list)

# file_list = [each for each in file_list if 'OCR' not in each]

# len(file_list)

In [9]:
file_list = ['RIA_Input/03-BCBS/risk/The role of household debt heterogeneity on consumption_ Evidence from Japanese household data.pdf']



# Clean Text & NER Extraction

In [10]:
import nltk
def tokenize(sentences):
    for sent in nltk.sent_tokenize(sentences.lower()):
        for word in nltk.word_tokenize(sent):                 
            yield word

print('OK !')

OK !


In [11]:
doc_list = []
for file_name in tqdm(file_list):
    doc = converter.convert(file_path = file_name, meta = None)
    long_text = doc[0]['content']
    text = nltk.Text(tkn for tkn in tokenize(long_text))
    text.collocations(num=10000000, window_size = 2)
    collocations = [" ".join(el) for el in list(text._collocations)]
    collocations = list(set(collocations))
    
    keyword = file_name.split('/')[-2]
    central_bank = file_name.split('/')[-3]
    
    for each_col in collocations:
        doc_dict = {}
        doc_dict['content'] = each_col
        doc_dict['content_type'] = 'text'
        doc_dict['meta'] = {}
        doc_dict['meta']['keyword'] = keyword
        doc_dict['meta']['central_bank'] = central_bank
        doc_list.append(doc_dict)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.24it/s]

precautionary saving; annual change; debt ratio; household debt; debt
heterogeneity; income elasticity; saving motive; low high; regression
model; old age; saving motives; monetary policy; negative income;
highly-indebted households; income change; international settlements;
year-fixed effects; categorical choices; standard errors; control
variables; 0.0 low; japanese households; dependent variable; robust
standard; statistical significance; june 2018; indicate statistical;
pps dataset; panel regression; estimation results; parameters study;
liquidity constraints; balance sheet; economic activity; residence
change; july 2018; public pension; zero otherwise; economic review;
household size; consumption behaviour; estimated using; high low;
22,485 5,837; jouchi nakajima; income changes; future work;
significantly positive; debt burdens; see also; job change; financial
stability; highly indebted; portfolio selection; upper quartile;
regression result; preference parameters; future unemplo

In [12]:
%%time

document_store.delete_documents()

document_store.write_documents(doc_list)

CPU times: user 22.7 ms, sys: 3.34 ms, total: 26.1 ms
Wall time: 2.95 s


In [13]:
er_retriever = EmbeddingRetriever(
   document_store=document_store,
   embedding_model="all-mpnet-base-v1",
   model_format="sentence_transformers",
   use_gpu = True,
)

document_store.update_embeddings(er_retriever)

torch.cuda.empty_cache()

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model all-mpnet-base-v1
INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 235 docs ...


Updating embeddings:   0%|          | 0/235 [00:00<?, ? Docs/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

# Question Generation

In [14]:
from haystack.pipelines import QuestionGenerationPipeline

question_generator = QuestionGenerator(model_name_or_path = 't5-base-e2e-qg')

question_generation_pipeline = QuestionGenerationPipeline(question_generator)

print('OK !')

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


OK !


In [15]:
doc_list = []
for idx, document in tqdm(enumerate(document_store_qg_input)):
    keyword = document.__dict__['meta']['keyword']
    central_bank = document.__dict__['meta']['central_bank']
    result = question_generation_pipeline.run(documents=[document])
    for each_qg in result['generated_questions'][0]['questions']:
        doc_dict = {}
        doc_dict['content'] = each_qg
        doc_dict['content_type'] = 'text'
        doc_dict['meta'] = {}
        doc_dict['meta']['keyword'] = keyword
        doc_dict['meta']['central_bank'] = central_bank
        doc_list.append(doc_dict)
    if idx == 2:
        break

2it [00:52, 26.40s/it]


In [16]:
%%time

document_store_qg.delete_documents()

document_store_qg.write_documents(doc_list)

CPU times: user 9.99 ms, sys: 2.76 ms, total: 12.8 ms
Wall time: 3.03 s


In [17]:
er_retriever = EmbeddingRetriever(
   document_store=document_store_qg,
   embedding_model="all-mpnet-base-v1",
   model_format="sentence_transformers",
   use_gpu = True,
)

document_store_qg.update_embeddings(er_retriever)

torch.cuda.empty_cache()

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model all-mpnet-base-v1
INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 38 docs ...


Updating embeddings:   0%|          | 0/38 [00:00<?, ? Docs/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]